In [1]:
import pickle
from timeit import default_timer as timer

import os
import platform
import tensorflow as tf
import keras;

# *print ("Current file path: {}".format(os.path.dirname(os.path.realpath(__file__))))
print ("Current Working Directory: {}".format(os.getcwd()))
print("Python: {}, TensorFlow:{}, Keras:{}".\
      format(platform.python_version(), tf.__version__, keras.__version__))

random_seed = 1



Using TensorFlow backend.


Current Working Directory: /home/endre/git/finance_ml/src
Python: 3.6.1, TensorFlow:1.2.1, Keras:2.0.5


# Load pickled Features, Labels and Name of each Feature-array

In [2]:
print ("De-pickling features, labels and feature/label names..")
load_start = timer()

# pickled = pickle.load(open("RangeNamesFeaturesAndLabels-mk1.pickle", "rb"))
# pickled = pickle.load(open("RangeNamesFeaturesAndLabels-smallSet-6stocks.pickle", "rb"))
# pickled = pickle.load(open("RangeNamesFeaturesAndLabels-smallSet-50stocks.pickle", "rb"))
pickled = pickle.load(open("RangeNamesFeaturesAndLabels-mediumset-unknownNumStocks.pickle", "rb"))
# pickled = pickle.load(open("RangeNamesFeaturesAndLabels-largeSet-1619stocks.pickle", "rb"))

load_millis = (timer()-load_start) * 1000
print ("De-pickle took {} ms".format(load_millis))

# {'rangeNames': rangeNames, 'features': features, 'labels': labels}
rangeNames = pickled['rangeNames']
features = pickled['features']
labels = pickled['labels']
print("\nRangeNames/features/labels len: {}/{}/{}".\
      format(len(rangeNames), len(features), len(labels)))
numFeatures = len(features[0])
print("\nNumber of Features for each feature-array: {}".format(numFeatures))
print("Number of Labels for each label-array: {}".format(len(labels[0])))

De-pickling features, labels and feature/label names..
De-pickle took 24434.17920399952 ms

RangeNames/features/labels len: 1008922/1008922/1008922

Number of Features for each feature-array: 54
Number of Labels for each label-array: 4


# Split set into Train and Test

In [3]:
import resource
import gc
def mem():
    print('Memory usage: %2.2f MB' % round(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0,1))
    
mem()
trainRangeNames = []
trainFeatures = []
trainLabels = []

testRangeNames = []
testFeatures = []
testLabels = []
mem()

gc.collect()
mem()

length = len(rangeNames)
for i in range(length):
    names = rangeNames[i]
    if (names[1] < '2016-01-01'):
        trainRangeNames.append(names)
        trainFeatures.append(features[i])
        trainLabels.append(labels[i])
    else:
        testRangeNames.append(names)
        testFeatures.append(features[i])
        testLabels.append(labels[i])

lenTrainFeatures = len(trainFeatures)

print("TRAIN RangeNames/features/labels len: {}/{}/{} - {:.2f}% of total".\
      format(len(trainRangeNames), lenTrainFeatures, len(trainLabels), (lenTrainFeatures / len(features)) * 100 ))
print("TEST RangeNames/features/labels len: {}/{}/{} - {:.2f}% of total".\
      format(len(testRangeNames), len(testFeatures), len(testLabels), (len(testFeatures) / len(features)) * 100 ))

print("\nTotal loaded features: {}, trainFeatures + testFeatures:{} - {:.4f}%".\
      format(len(features), lenTrainFeatures + len(testFeatures), ((lenTrainFeatures + len(testFeatures)) / len(features)) * 100))

Memory usage: 7892.30 MB
Memory usage: 7892.30 MB
Memory usage: 7892.30 MB
TRAIN RangeNames/features/labels len: 939830/939830/939830 - 93.15% of total
TEST RangeNames/features/labels len: 69092/69092/69092 - 6.85% of total

Total loaded features: 1008922, trainFeatures + testFeatures:1008922 - 100.0000%


In [4]:
# One-hot the labels

# 0:  5 days
# 1: 10 days
# 2: 15 days
# 3: 20 days
label_to_use = 3

good = [1,0]
bad =  [0,1]

trainLabels_onehot = [good if x[label_to_use] > 0 else bad  for x in trainLabels]
testLabels_onehot = [good if x[label_to_use] > 0 else bad  for x in testLabels]
print("One-hotted ok.")

sum_trainLabels = [sum(i) for i in zip(*trainLabels_onehot)]
sum_testLabels = [sum(i) for i in zip(*testLabels_onehot)]
print("Train labels [good, bad]: {} -> [{:.2f}%, {:.2f}%]".\
      format(sum_trainLabels, (sum_trainLabels[0] / len(trainLabels)) * 100, (sum_trainLabels[1] / len(trainLabels)) * 100))
print("Test labels  [good, bad]: {} -> [{:.2f}%, {:.2f}%]".\
      format(sum_testLabels, (sum_testLabels[0] / len(testLabels)) * 100, (sum_testLabels[1] / len(testLabels)) * 100))

One-hotted ok.
Train labels [good, bad]: [505724, 434106] -> [53.81%, 46.19%]
Test labels  [good, bad]: [39999, 29093] -> [57.89%, 42.11%]


# !! RUN FROM HERE WHEN ONLY TWEAKING NETWORK !!

In [63]:
from sklearn.utils import shuffle
import numpy as np

currentTrainPos = 0
current_shuffle = 0
shuff_trainFeatures = []
shuff_trainLabels_onehot = []
def getTrainMiniBatch(miniBatchSize):
    global currentTrainPos, current_shuffle
    global shuff_trainFeatures, shuff_trainLabels_onehot
    # If we'll overflow the features list with this mini_batch, then reset the start position
    if (currentTrainPos + miniBatchSize > lenTrainFeatures):
        currentTrainPos = 0

    # If the start position is 0, then run shuffling of features and labels.
    if (currentTrainPos == 0):
        print("Shuffling training features and labels - {}..".format(current_shuffle))
        shuff_trainFeatures, shuff_trainLabels_onehot = shuffle(trainFeatures, trainLabels_onehot, random_state=current_shuffle)
        current_shuffle += 1

    # Pick out the desired mini_batch size.
    start = currentTrainPos
    end = currentTrainPos + miniBatchSize
    # .. store this mini_batch end pos as the next mini_batch start pos.
    currentTrainPos = end
    
    return np.array(shuff_trainFeatures[start:end]) / 100.0, np.array(shuff_trainLabels_onehot[start:end])
    # return shuff_trainFeatures[start:end], shuff_trainLabels_onehot[start:end]

# Set up Neural Network

In [64]:
n1=800
n2=1600
n3=800
n4=600
out=2
epochs = 100 # Not explicitly used, as we count mini_batches
mini_batch_size = 128
learning_rate = 0.0001  # 0.0001 is good, but maybe use higher learning rate when dropout'ing..
keep_prob_in_train = 1
keep_prob_hid_train = 1
transfer_function = tf.nn.relu
l2_regularization_beta = 0.0000 # This seemed good, but currently having all softmaxed_test_Y to [ 0.53961992,  0.46038002]: 0.025

#========
mini_batches_per_epoch = int(lenTrainFeatures / mini_batch_size)
total_mini_batches = mini_batches_per_epoch * epochs

print("Mini batches per epoch: trainFeats {} / mini_batch_size {} = {}".format(lenTrainFeatures, mini_batch_size, mini_batches_per_epoch))
print("Total mini batches: mini_batches_per_epoch {} * epochs {} = {}".format(mini_batches_per_epoch, epochs, total_mini_batches))

# Reset the default graph, so as to chuck out existing variables
tf.reset_default_graph()

with tf.variable_scope("input"):
    X = tf.placeholder(tf.float32, shape=[None, numFeatures], name="X")
    keep_prob_in = tf.placeholder(tf.float32, name="keep_prob")
    # X_drop = tf.nn.dropout(X, keep_prob_in)
    labels = tf.placeholder(tf.float32, shape=[None, out], name="labels")

with tf.variable_scope("hidden_layers_common"):
    keep_prob_hid = tf.placeholder(tf.float32, name="keep_prob")

def weight_variable(name, shape):
    # return tf.get_variable(name, initializer=tf.glorot_uniform_initializer(seed=random_seed), shape=shape)
    return tf.get_variable(name, initializer=tf.contrib.layers.xavier_initializer(), shape=shape)
    
def bias_variable(name, shape):
    # return tf.get_variable(name, initializer=tf.glorot_uniform_initializer(seed=random_seed), shape=shape)
    # return tf.get_variable(name, initializer=tf.contrib.layers.xavier_initializer(), shape=shape)
    return tf.get_variable(name, initializer=tf.constant_initializer(0.1), shape=shape)

with tf.variable_scope("layer1"):
    w1 = weight_variable("weight", [numFeatures, n1])
    b1 = bias_variable("bias", [n1])
    l1 = tf.matmul(X, w1) + b1
    l1 = transfer_function(l1)
    # l1 = tf.nn.dropout(l1, keep_prob_hid)

with tf.variable_scope("layer2"):
    w2 = weight_variable("weight", [n1, n2])
    b2 = bias_variable("bias", [n2])
    l2 = tf.matmul(l1, w2) + b2
    l2 = transfer_function(l2)
    # l2 = tf.nn.dropout(l2, keep_prob_hid)

with tf.variable_scope("layer3"):
    w3 = weight_variable("weight", [n2, n3])
    b3 = bias_variable("bias", [n3])
    l3 = tf.matmul(l2, w3) + b3
    l3 = transfer_function(l3)
    # l3 = tf.nn.dropout(l3, keep_prob_hid)

with tf.variable_scope("layer4"):
    w4 = weight_variable("weight", [n3, n4])
    b4 = bias_variable("bias", [n4])
    l4 = tf.matmul(l3, w4) + b4
    l4 = transfer_function(l4)
    # l4 = tf.nn.dropout(l4, keep_prob_hid)

with tf.variable_scope("output"):
    wy = weight_variable("weight", [n4, out])
    by = bias_variable("bias", [out])
    Y = tf.matmul(l4, wy) + by

# :: Loss function with L2 Regularization
l2_regularizer = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3) \
               + tf.nn.l2_loss(wy)
# NOTE: L1 regularizer: tf.reduce_sum(tf.abs(tensor)) instead of tf.nn.l2_loss(tensor)
# Read here: https://github.com/tensorflow/models/blob/master/inception/inception/slim/losses.py
# Max-norm:
# https://stackoverflow.com/questions/37801832/how-can-i-implement-max-norm-constraints-in-an-mlp-in-tensorflow
# Hmm..: https://stackoverflow.com/questions/34934303/renormalize-weight-matrix-using-tensorflow

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=labels) +\
                      l2_regularization_beta * l2_regularizer)

# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y, labels=labels))

# :: Optimizer/Trainer
# optimizer = tf.train.AdadeltaOptimizer(learning_rate)
optimizer = tf.train.AdamOptimizer(learning_rate)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

# Define Test/Evaluate: Accuracy: Fraction right predictions
argmaxed_Y = tf.argmax(Y, 1)
correct_prediction = tf.equal(argmaxed_Y, tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Make Softmax node, to use for evaluation
softmaxed_Y = tf.nn.softmax(Y)

print("Network set up.")

Mini batches per epoch: trainFeats 939830 / mini_batch_size 128 = 7342
Total mini batches: mini_batches_per_epoch 7342 * epochs 100 = 734200
Network set up.


# Train Neural Network

In [ ]:
check_length = len(testFeatures)
check_trainFeatures, check_trainLabels_onehot = shuffle(trainFeatures, trainLabels_onehot, random_state=42)
check_trainFeatures = check_trainFeatures[:check_length]
check_trainLabels_onehot = check_trainLabels_onehot[:check_length]

sess = tf.Session()

# Initialize TensorFlow variables in Session
sess.run(tf.global_variables_initializer())

train_argmaxed_labels = np.argmax(check_trainLabels_onehot, axis=1)
test_argmaxed_labels = np.argmax(testLabels_onehot, axis=1)
_, train_label_counts = np.unique(train_argmaxed_labels, return_counts=True)
_, test_label_counts = np.unique(test_argmaxed_labels, return_counts=True)

import pandas as pd
from IPython.display import display_html
def printStats(what):
    display_html("<hr />")
    train_cost, train_accuracy, train_argmaxed_Y = sess.run([loss, accuracy, argmaxed_Y], 
                                       feed_dict={X: check_trainFeatures,
                                                  labels: check_trainLabels_onehot,
                                                  keep_prob_in: 1.0,
                                                  keep_prob_hid: 1.0})
    test_cost, test_accuracy, test_argmaxed_Y = sess.run([loss, accuracy, argmaxed_Y], 
                                     feed_dict={X: testFeatures,
                                                labels: testLabels_onehot,
                                                keep_prob_in: 1.0,
                                                keep_prob_hid: 1.0})
    _, train_Y_counts = np.unique(train_argmaxed_Y, return_counts=True)
    _, test_Y_counts = np.unique(test_argmaxed_Y, return_counts=True)
    print("{}\n  TRAIN cost/acc:{:.4f}/{:.4f}% {}/{}, TEST cost/acc:{:.4f}/{:.4f}% {}/{}"\
          .format(what, train_cost, train_accuracy*100, train_Y_counts, train_label_counts,
                  test_cost, test_accuracy*100, test_Y_counts, test_label_counts))

    pds_train_labels = pd.Series(train_argmaxed_labels, name='Labels')
    pds_train_Y = pd.Series(train_argmaxed_Y, name='Y train')
    df_confusion_train = pd.crosstab(pds_train_labels, pds_train_Y, margins=True)

    pds_test_labels = pd.Series(test_argmaxed_labels, name='Labels')
    pds_test_Y = pd.Series(test_argmaxed_Y, name='Y test')
    df_confusion_test = pd.crosstab(pds_test_labels, pds_test_Y, margins=True)
    
    html_str = df_confusion_train.to_html() + df_confusion_test.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)
    

printStats("BEFORE Training")

BEFORE Training
  TRAIN cost/acc:2.4107/46.2572% [ 1515 67577]/[37271 31821], TEST cost/acc:1.6336/42.4260% [ 1796 67296]/[39999 29093]


Y train,0,1,All
Labels,,,
0,827,36444,37271
1,688,31133,31821
All,1515,67577,69092
Y test,0,1,All
Labels,,,
0,1008,38991,39999
1,788,28305,29093
All,1796,67296,69092


In [ ]:
time_start = timer()
epoch = 0
for mini_batch in range(1, total_mini_batches+1):
    batch_inputs, batch_labels = getTrainMiniBatch(mini_batch_size)
    sess.run([train, accuracy], feed_dict={X: batch_inputs, 
                                           labels: batch_labels,
                                           keep_prob_in: keep_prob_in_train,
                                           keep_prob_hid: keep_prob_hid_train})
    if ((epoch == 0) and (mini_batch % 100 == 0) and (mini_batch <= 1000)):
        printStats(' ... progress: In "Epoch" {} of {}, mini batch {} of {}'.format(epoch, epochs, mini_batch, total_mini_batches))
    if (((epoch > 0) or (mini_batch > 1000)) and (mini_batch % 500 == 0)):
        print(' ... progress: In "Epoch" {} of {}, mini batch {} of {}'.format(epoch, epochs, mini_batch, total_mini_batches))
        
    if (mini_batch % mini_batches_per_epoch == 0):
        epoch += 1
        printStats('\nAfter "Epoch" {} of {}, mini batch {} of {}'.format(epoch, epochs, mini_batch, total_mini_batches))
        
training_time = timer()-time_start
printStats("\nFINISHED")
print("TRAINING took {} seconds.".format(training_time))

Shuffling training features and labels - 0..
 ... progress: In "Epoch" 0 of 100, mini batch 100 of 734200
  TRAIN cost/acc:1.6950/53.9324% [68190   902]/[37271 31821], TEST cost/acc:1.2255/57.6883% [68137   955]/[39999 29093]


Y train,0,1,All
Labels,,,
0,36816,455,37271
1,31374,447,31821
All,68190,902,69092
Y test,0,1,All
Labels,,,
0,39451,548,39999
1,28686,407,29093
All,68137,955,69092


 ... progress: In "Epoch" 0 of 100, mini batch 200 of 734200
  TRAIN cost/acc:1.2725/52.6573% [39491 29601]/[37271 31821], TEST cost/acc:0.9336/49.0419% [30739 38353]/[39999 29093]


Y train,0,1,All
Labels,,,
0,22026,15245,37271
1,17465,14356,31821
All,39491,29601,69092
Y test,0,1,All
Labels,,,
0,17765,22234,39999
1,12974,16119,29093
All,30739,38353,69092


 ... progress: In "Epoch" 0 of 100, mini batch 300 of 734200
  TRAIN cost/acc:1.2284/50.2475% [25016 44076]/[37271 31821], TEST cost/acc:1.0797/48.1604% [20198 48894]/[39999 29093]


...

Shuffling training features and labels - 27..
 ... progress: In "Epoch" 27 of 100, mini batch 198500 of 734200
 ... progress: In "Epoch" 27 of 100, mini batch 199000 of 734200
 ... progress: In "Epoch" 27 of 100, mini batch 199500 of 734200
 ... progress: In "Epoch" 27 of 100, mini batch 200000 of 734200
 ... progress: In "Epoch" 27 of 100, mini batch 200500 of 734200
 ... progress: In "Epoch" 27 of 100, mini batch 201000 of 734200
 ... progress: In "Epoch" 27 of 100, mini batch 201500 of 734200
 ... progress: In "Epoch" 27 of 100, mini batch 202000 of 734200
 ... progress: In "Epoch" 27 of 100, mini batch 202500 of 734200
 ... progress: In "Epoch" 27 of 100, mini batch 203000 of 734200
 ... progress: In "Epoch" 27 of 100, mini batch 203500 of 734200
 ... progress: In "Epoch" 27 of 100, mini batch 204000 of 734200
 ... progress: In "Epoch" 27 of 100, mini batch 204500 of 734200
 ... progress: In "Epoch" 27 of 100, mini batch 205000 of 734200
 ... progress: In "Epoch" 27 of 100, mini ba

Y train,0,1,All
Labels,,,
0,20266,17005,37271
1,10861,20960,31821
All,31127,37965,69092
Y test,0,1,All
Labels,,,
0,18510,21489,39999
1,13614,15479,29093
All,32124,36968,69092


Shuffling training features and labels - 28..
 ... progress: In "Epoch" 28 of 100, mini batch 206000 of 734200
 ... progress: In "Epoch" 28 of 100, mini batch 206500 of 734200
 ... progress: In "Epoch" 28 of 100, mini batch 207000 of 734200
 ... progress: In "Epoch" 28 of 100, mini batch 207500 of 734200
 ... progress: In "Epoch" 28 of 100, mini batch 208000 of 734200
 ... progress: In "Epoch" 28 of 100, mini batch 208500 of 734200
 ... progress: In "Epoch" 28 of 100, mini batch 209000 of 734200
 ... progress: In "Epoch" 28 of 100, mini batch 209500 of 734200
 ... progress: In "Epoch" 28 of 100, mini batch 210000 of 734200
 ... progress: In "Epoch" 28 of 100, mini batch 210500 of 734200
 ... progress: In "Epoch" 28 of 100, mini batch 211000 of 734200
 ... progress: In "Epoch" 28 of 100, mini batch 211500 of 734200
 ... progress: In "Epoch" 28 of 100, mini batch 212000 of 734200
 ... progress: In "Epoch" 28 of 100, mini batch 212500 of 734200

After "Epoch" 29 of 100, mini batch 212918 

Y train,0,1,All
Labels,,,
0,19129,18142,37271
1,9897,21924,31821
All,29026,40066,69092
Y test,0,1,All
Labels,,,
0,16933,23066,39999
1,12731,16362,29093
All,29664,39428,69092


Shuffling training features and labels - 29..
 ... progress: In "Epoch" 29 of 100, mini batch 213000 of 734200
 ... progress: In "Epoch" 29 of 100, mini batch 213500 of 734200
 ... progress: In "Epoch" 29 of 100, mini batch 214000 of 734200
 ... progress: In "Epoch" 29 of 100, mini batch 214500 of 734200
 ... progress: In "Epoch" 29 of 100, mini batch 215000 of 734200
 ... progress: In "Epoch" 29 of 100, mini batch 215500 of 734200
 ... progress: In "Epoch" 29 of 100, mini batch 216000 of 734200
 ... progress: In "Epoch" 29 of 100, mini batch 216500 of 734200
 ... progress: In "Epoch" 29 of 100, mini batch 217000 of 734200
 ... progress: In "Epoch" 29 of 100, mini batch 217500 of 734200
 ... progress: In "Epoch" 29 of 100, mini batch 218000 of 734200
 ... progress: In "Epoch" 29 of 100, mini batch 218500 of 734200
 ... progress: In "Epoch" 29 of 100, mini batch 219000 of 734200
